In [1]:
# 라이브러리 불러오기
from selenium import webdriver
from bs4 import BeautifulSoup
import time

from tqdm.notebook import tqdm # 상태진행바
import pandas as pd

from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.by import By

import json

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.DataFrame(columns = ["사용자", "가게명", "리뷰"])
df

,사용자,가게명,리뷰


In [3]:
df_read = pd.read_csv('/data/사용자의 리뷰페이지 데이터 합치기 완료 최종 - 중복제거 완료.csv')
df_read

,닉네임,링크,가게명,_merge
0,first Katz,https://m.place.naver.com/my/5d7218388f87a842b...,하이레,both
1,kan****,https://m.place.naver.com/my/5e2985118f87a842b...,하이레,both
2,희희맛,https://m.place.naver.com/my/5bbb69b75473a64c6...,하이레,both
3,부천후후,https://m.place.naver.com/my/5d08b4bcaeba04d9e...,하이레,both
4,이동네먹짱,https://m.place.naver.com/my/5f061920de46936b6...,하이레,both
...,...,...,...,...
7254,초오로오오오옹,https://m.place.naver.com/my/5bbcd2645f0e649c0...,주공 장위직영,right_only
7255,아홉 9,https://m.place.naver.com/my/5ce0bcfb440a9218c...,주공 장위직영,right_only
7256,민지3156,https://m.place.naver.com/my/5f3dbffc7ecfbfb7b...,주공 장위직영,right_only
7257,BunnY,https://m.place.naver.com/my/6054a418bd4ce1d8f...,디델리 서울 광운대점,right_only


# 1. 브라우저 열고 접속

In [4]:
# 브라우저 열기
browser = webdriver.Chrome()

In [12]:
for num in range(3582, len(df_read)):
    browser.get(df_read['링크'].iloc[num])
    user = df_read['닉네임'][num]

    time.sleep(2)
    
    # 사진/영상 리뷰만 클릭 해제
    soup = BeautifulSoup(browser.page_source, 'html.parser')

    time.sleep(3)

    cnt_compare1 = soup.select('._5RYLm7._6IqZAz > em')[0].text.replace(",","")
    print(f'{user}가 적은 리뷰의 총 개수는 {cnt_compare1}')
    
#     if soup.select('._6r5HI8'):
#         browser.find_element(By.CSS_SELECTOR, '._3P1WRE').click()
#     else:
#         try:
#             cnt_compare2 = soup.select('.ciTfFP > ._5RYLm7._5FMBWE > .C7R4SD')[0].text.replace(",","")
#         except:
#             cnt_compare2 = str(len(soup.select('._3HPEOr')))

#         if int(cnt_compare1) != int(cnt_compare2):
#             browser.find_element(By.CSS_SELECTOR, '._3P1WRE').click()
            
    if soup.select('._3HPEOr'):
        pass
    else:
        browser.find_element(By.CSS_SELECTOR, '._3P1WRE').click()
        
    time.sleep(3)   
    
    # 첫번째 리뷰 클릭
    # browser.find_elements(By.CSS_SELECTOR, '._3HPEOr > ._5RYLm7')[0].click()
    browser.find_elements(By.CSS_SELECTOR, '._5RYLm7.bEchjN')[0].click()

    time.sleep(3)

#     스크롤
    last_page_height = browser.execute_script("return document.body.scrollHeight")
    print(f'첫번째 높이는 {last_page_height}')
    
    for _ in tqdm(range(50)):
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        new_page_height = browser.execute_script("return document.body.scrollHeight")
        time.sleep(1)
        print(f'새로 가져온 높이는  {new_page_height}')
        
        if new_page_height == last_page_height:
            break
        else:
            last_page_height = new_page_height
#             print(f'업데이트한 높이는 {last_page_height}')

#     # Find the element with class "ptr__children"
#     scrollable_element = WebDriverWait(browser, 10).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "ptr__children"))
#     )

#     # Scroll the element
#     scroll_element_to_bottom(browser, scrollable_element)

#     body = browser.find_element(By.CSS_SELECTOR, 'body') # send_keys()메서드 사용을 위한 body가져오기
#     for _ in tqdm(range(50)):
#         body.send_keys(Keys.PAGE_DOWN) # 페이지 다운 키를  20회 반복한다.
#         time.sleep(0.1)

#         time.sleep(1)
    
    # 리뷰에서 더보기를 눌러야하는 것들 미리 누름
    add_buttons = browser.find_elements(By.CSS_SELECTOR, '._5RYLm7._7WkVWf')
    print(f'더보기 버튼을 눌러야 하는 갯수: {len(add_buttons)}')

    time.sleep(1)

    for add_button in tqdm(add_buttons):
        try:
            add_button.click()
        except:
            browser.execute_script("arguments[0].click();", add_button)
        time.sleep(0.2)

    print('더보기 버튼 클릭 완료')
    
    newSoup = BeautifulSoup(browser.page_source, 'html.parser')

    time.sleep(3)
    
    person_review_all_div_list = newSoup.select('._898uc8')

    print(f'페이지 스크롤한 다음 페이지에서 보여지는 총 리뷰 리스트 개수는 {len(person_review_all_div_list)}')
    
    if int(cnt_compare1) == len(person_review_all_div_list):
        print('모든 리뷰를 가지고 왔음')
    else:
        print('양이 많아서 일부 리뷰만 가지고 옴')
        
    person_review_cnt = 0
    user_review = []

    for person_review_all_div in person_review_all_div_list:

        if person_review_all_div.select('._3n2arC'):
            restaurant_name = person_review_all_div.select('._5RYLm7.dmNX1z')[0].text
            real_review = person_review_all_div.select('._3n2arC')[0].text.strip()
            person_review_cnt += 1

            data = [user, restaurant_name, real_review]
            user_review.append(data)
            
    print(f'{num+1}번째 사람의 {person_review_cnt}개의 리뷰를 가져왔음.')
    print('\n')
    
    if user_review:  # Only create and append DataFrame if there are reviews
        user_review_df = pd.DataFrame(user_review, columns=["사용자", "가게명", "리뷰"])
        df = df.append(user_review_df, ignore_index=True)
    else:
        print(f'{num+1}번째 사람의 리뷰를 가져오지 못했습니다.')

맛있구마27가 적은 리뷰의 총 개수는 1054
첫번째 높이는 8545


  0%|          | 0/50 [00:00<?, ?it/s]

새로 가져온 높이는  16082
새로 가져온 높이는  18441
새로 가져온 높이는  18441
더보기 버튼을 눌러야 하는 갯수: 0


0it [00:00, ?it/s]

더보기 버튼 클릭 완료
페이지 스크롤한 다음 페이지에서 보여지는 총 리뷰 리스트 개수는 24
양이 많아서 일부 리뷰만 가지고 옴
3583번째 사람의 16개의 리뷰를 가져왔음.


출출해윤51가 적은 리뷰의 총 개수는 793
첫번째 높이는 9626


...

  0%|          | 0/50 [00:00<?, ?it/s]

새로 가져온 높이는  16178
새로 가져온 높이는  24303
새로 가져온 높이는  32059
새로 가져온 높이는  38799
새로 가져온 높이는  46409
새로 가져온 높이는  47900
새로 가져온 높이는  47900
더보기 버튼을 눌러야 하는 갯수: 0


0it [00:00, ?it/s]

더보기 버튼 클릭 완료
페이지 스크롤한 다음 페이지에서 보여지는 총 리뷰 리스트 개수는 63
양이 많아서 일부 리뷰만 가지고 옴
3697번째 사람의 22개의 리뷰를 가져왔음.




IndexError: list index out of range

In [10]:
df.to_csv('/data/2979명의 리뷰 데이터.csv', index = False)